In [1]:
import os
import re
import json
import pandas as pd
import numpy as np
# from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
# from sklearn import metrics
# import nltk
# from nltk.stem import wordnet, SnowballStemmer
# from nltk.tokenize import word_tokenize
# from nltk.corpus import stopwords
# import operator


In [ ]:
# class DataLoader(object):
#     def __init__(self):
#         self.movies_n_series, self.ratings = self.load_imdb()
#         self.wiki = self.load_wiki()

#     @staticmethod
#     def open_json(full_file_path):
#         with open(full_file_path, 'r') as jfile:
#             return json.load(jfile)

#     def load_from_path(self, path_for_data_files):
#         full_path_file_names = list(map(lambda p: os.path.join(path_for_data_files, p), os.listdir(path_for_data_files)))
#         only_valid_files = list(filter(lambda file: '.json' in file and not ('!' in file or '?' in file),
#                                        full_path_file_names))
#         return list(map(self.open_json, only_valid_files))

#     def load_imdb(self):
#         movies_n_series = []
#         ratings = []
#         path_for_movie_jsons = 'raw_data'
#         for movie_details in self.load_from_path(path_for_movie_jsons):
#                 if 'Ratings' in movie_details:
#                     for rating in movie_details['Ratings']:
#                         rating['imdbID'] = movie_details['imdbID']

#                     ratings += movie_details.pop('Ratings')
#                 movies_n_series.append(movie_details)

#         return pd.DataFrame(movies_n_series), pd.DataFrame(ratings)

#     def load_wiki(self):
#         path_for_wiki_jsons = os.path.join('raw_data', 'wiki_data')
#         wiki = self.load_from_path(path_for_wiki_jsons)

#         return pd.DataFrame(wiki)


In [2]:
def open_json(full_file_path):
    with open(full_file_path, 'r') as jfile:
        x = json.load(jfile)
    return x

In [4]:
file_name = 'tt0021814.json'
imdb_data = open_json(os.path.join('raw_imdb_data', file_name))
wiki_data = open_json(os.path.join('raw_wiki_data', file_name))

In [5]:
imdb_data

{'Title': 'Dracula',
 'Year': '1931',
 'Rated': 'NOT RATED',
 'Released': '14 Feb 1931',
 'Runtime': '75 min',
 'Genre': 'Fantasy, Horror',
 'Director': 'Tod Browning',
 'Writer': 'Bram Stoker (by), Hamilton Deane (from the play adapted by), John L. Balderston (from the play adapted by), Garrett Fort (play)',
 'Actors': 'Bela Lugosi, Helen Chandler, David Manners, Dwight Frye',
 'Plot': 'The ancient vampire Count Dracula arrives in England and begins to prey upon the virtuous young Mina.',
 'Language': 'English, Hungarian, Latin',
 'Country': 'USA',
 'Awards': '3 wins & 3 nominations.',
 'Poster': 'https://ia.media-imdb.com/images/M/MV5BZDY2ODZhZWQtNDk0ZS00OGE4LWE4NjAtZjE5MTJhMjExMTRjXkEyXkFqcGdeQXVyNjc1NTYyMjg@._V1_SX300.jpg',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.6/10'},
  {'Source': 'Rotten Tomatoes', 'Value': '91%'}],
 'Metascore': 'N/A',
 'imdbRating': '7.6',
 'imdbVotes': '38,346',
 'imdbID': 'tt0021814',
 'Type': 'movie',
 'DVD': '28 Aug 2001',
 'BoxOffi

In [6]:
wiki_data

{'text': '\n\nDracula is a 1931 American pre-Code vampire-horror film directed by Tod Browning and starring Bela Lugosi as Count Dracula. The film was produced by Universal and is based on the 1924 stage play Dracula by Hamilton Deane and John L. Balderston, which in turn is loosely based on the novel Dracula by Bram Stoker.',
 'wiki_page_id': 1046559,
 'imdb_id': 'tt0021814'}

In [12]:
from bs4 import BeautifulSoup
def cleanText(text):
    text = BeautifulSoup(text, "lxml").text
    text = re.sub(r'\|\|\|', r' ', text) 
    text = re.sub(r'http\S+', r'<URL>', text)
    text = text.lower()
    text = text.replace('x', '')
    return text

In [11]:
import nltk
from nltk.corpus import stopwords
def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens


In [10]:
full_text = imdb_data['Plot'] + ' ' + wiki_data['text']
full_text

'The ancient vampire Count Dracula arrives in England and begins to prey upon the virtuous young Mina. \n\nDracula is a 1931 American pre-Code vampire-horror film directed by Tod Browning and starring Bela Lugosi as Count Dracula. The film was produced by Universal and is based on the 1924 stage play Dracula by Hamilton Deane and John L. Balderston, which in turn is loosely based on the novel Dracula by Bram Stoker.'

In [25]:
x = re.sub(r'\n', '', full_text)
x

'The ancient vampire Count Dracula arrives in England and begins to prey upon the virtuous young Mina. Dracula is a 1931 American pre-Code vampire-horror film directed by Tod Browning and starring Bela Lugosi as Count Dracula. The film was produced by Universal and is based on the 1924 stage play Dracula by Hamilton Deane and John L. Balderston, which in turn is loosely based on the novel Dracula by Bram Stoker.'

In [21]:
import string
'|'.join(list(string.punctuation))

'!|"|#|$|%|&|\'|(|)|*|+|,|-|.|/|:|;|<|=|>|?|@|[|\\|]|^|_|`|{|||}|~'

In [45]:
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

def token_text(full_text, remove_stop_words=True, remove_punctuations=True, remove_if_not_alpha=True, stem_word=False):
    porter = PorterStemmer()

    stop_words = set(stopwords.words('english'))
    punctuations = list(string.punctuation)

    tokenize_words = []
    for sentence in nltk.sent_tokenize(full_text.lower()):
        for word in nltk.word_tokenize(sentence):
            if word in stop_words and remove_stop_words == True:
                continue
            if word in punctuations and remove_punctuations == True:
                continue
            if not word.isalpha() and remove_if_not_alpha == True:  # if not all characters in the string are alphabetic
                continue
            if stem_word:
                word = porter.stem(word)

            tokenize_words.append(word)
    return tokenize_words
                             
                             
token_text(full_text, remove_stop_words=False, remove_punctuations=False, remove_if_not_alpha=False, stem_word=False)

['the',
 'ancient',
 'vampire',
 'count',
 'dracula',
 'arrives',
 'in',
 'england',
 'and',
 'begins',
 'to',
 'prey',
 'upon',
 'the',
 'virtuous',
 'young',
 'mina',
 '.',
 'dracula',
 'is',
 'a',
 '1931',
 'american',
 'pre-code',
 'vampire-horror',
 'film',
 'directed',
 'by',
 'tod',
 'browning',
 'and',
 'starring',
 'bela',
 'lugosi',
 'as',
 'count',
 'dracula',
 '.',
 'the',
 'film',
 'was',
 'produced',
 'by',
 'universal',
 'and',
 'is',
 'based',
 'on',
 'the',
 '1924',
 'stage',
 'play',
 'dracula',
 'by',
 'hamilton',
 'deane',
 'and',
 'john',
 'l.',
 'balderston',
 ',',
 'which',
 'in',
 'turn',
 'is',
 'loosely',
 'based',
 'on',
 'the',
 'novel',
 'dracula',
 'by',
 'bram',
 'stoker',
 '.']